# **Course project on Technologies and Infrastructure for Big Data**
part I : image collecting from flickr


Project topic: image segmentation with data collection from www.flickr.com database 

Project executors:

Ma Cheng Yuan , ISU 372818 , J4133c

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=f40e4a9780b3f376c2cc7ff5bb5fab520184ad05e26c31481dce13cc2e222ea3
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
pip install flickrapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.5 MB/s eta 0:00:00


In [3]:
import pyspark.pandas as ps
import flickrapi
from flickrapi import FlickrAPI
import os
import warnings

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


#Download image from flick with flickrapi

- image segmentation , all_species = ['`**blue jay**', '**northern cardinal**', '**american goldfinch**`'] , each specie with 20 image 

Before we start parsing data, we have received key and secret code to connect to the API. You can get your own key in the Developers section of www.flickr.com or use ours.

In [4]:
from flickrapi import FlickrAPI
from pyspark.sql import SparkSession
from pyspark.sql.types import *

API_KEY = '513e24d8fc90dd4f767670d583ba1e41'
API_SECRET = '975b054fcd3c3371'
api = flickrapi.FlickrAPI(api_key=API_KEY, secret=API_SECRET, format='parsed-json')


def get_photos(image_tag):
    extras = ','.join(SIZES)
    flickr = FlickrAPI(API_KEY, API_SECRET)
    photos = flickr.walk(text=image_tag,
                            extras="url_c",  # get the url for the original size image
                            privacy_filter=1,  # search only for public photos
                            per_page=500,
                            sort='relevance')
    
  
    
    return photos

def get_url(photo):
      url = photo.get('url_c')     
    
      if url:  # if url is None try with the next size
          return url

def get_urls(image_tag, max):
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            url = get_url(photo)  # get preffered size url
            if url:
                urls.append(url)
                counter += 1
            # if no url for the desired sizes then try with the next photo
        else:
            break

    return urls

In [5]:
import requests
import os
import sys

def create_folder(path):
    if not os.path.isdir(path):
        os.makedirs(path)

def download_images(urls, path):
    create_folder(path)  # makes sure path exists
    local_storage = []
    for url in urls:
        image_name = url.split("/")[-1]
        image_path = os.path.join(path, image_name)
        
        if not os.path.isfile(image_path):  # ignore if already downloaded
            response=requests.get(url,stream=True)

            with open(image_path,'wb') as outfile:
                outfile.write(response.content)
        local_storage.append(image_path)
    return local_storage

#explanation to argument api.photo.search method :


*   `text` : description of image
*   `sort` : flickr's built-in sorting method
*   `media`: data type (image,video)
*   `extras`: image metadata. We chose url_c, which will return url of the image thumbnail with a size of 800 pixels on the long side
*   `per_page`:number of images per query response page
*   `page`:page number of query responses







In [6]:
from pyspark.sql.readwriter import DataFrameReader
import os
import time

all_species = ['blue jay', 'northern cardinal', 'american goldfinch']
 
def download(urls,specie):
    fotos = []
    local_storage = []
    
    
    print('Getting urls for', specie)
    urls = urls

    print('Downlaing images for', specie)
    path = os.path.join('data', specie)
    
    
    local_storage = download_images(urls, path)
    
    return local_storage


if __name__=='__main__':
  df = ps.DataFrame()
  labels = []
  size = 20
  flickr = FlickrAPI(API_KEY, API_SECRET)
  for label in all_species :
    page = 1
    dataframe = ps.DataFrame()
    print(label)    
    while(len(dataframe) < size):
      photos = api.photos.search(text=label, sort='relevance', media='photos',
                    extras='url_c', per_page=size, page=page)['photos']
      dataframe_specie = ps.DataFrame(photos['photo'])
      dataframe = ps.concat([dataframe, dataframe_specie], ignore_index=True)
      page = page + 1
  
    dataframe['label'] = [all_species.index(label)] * len(dataframe)
    dataframe = dataframe.dropna()
    urls = dataframe['url_c'].tolist()
    local_storage = download(urls,label)
    print(len(local_storage))
    print(len(dataframe))
    dataframe['root'] = local_storage
    df = ps.concat([df, dataframe], ignore_index=True)
  
 

blue jay
Getting urls for blue jay
Downlaing images for blue jay
18
18
northern cardinal
Getting urls for northern cardinal
Downlaing images for northern cardinal
20
20
american goldfinch
Getting urls for american goldfinch
Downlaing images for american goldfinch
20
20


In [7]:
print("there are total {} downloaded images".format(len(df)))


there are total 58 downloaded images


convert files with images into zip files and download it to create masked images (label images) using VGG image annotator 

https://annotate.officialstatistics.org/



In [8]:
!zip -r /content/data.zip /content/data

  adding: content/data/ (stored 0%)
  adding: content/data/northern cardinal/ (stored 0%)
  adding: content/data/northern cardinal/50962862286_5ce1a3a684_c.jpg (deflated 1%)
  adding: content/data/northern cardinal/52134463708_2022150516_c.jpg (deflated 1%)
  adding: content/data/northern cardinal/52443481589_e17826827a_c.jpg (deflated 1%)
  adding: content/data/northern cardinal/31250549107_86018696e6_c.jpg (deflated 1%)
  adding: content/data/northern cardinal/51790286079_c6aa2d0fb2_c.jpg (deflated 1%)
  adding: content/data/northern cardinal/52642590670_04bb6b345e_c.jpg (deflated 0%)
  adding: content/data/northern cardinal/51898327672_ab7d9107cb_c.jpg (deflated 0%)
  adding: content/data/northern cardinal/52702825314_ba9c4e53d0_c.jpg (deflated 5%)
  adding: content/data/northern cardinal/52439266779_6711effd90_c.jpg (deflated 1%)
  adding: content/data/northern cardinal/30376042413_1a15900032_c.jpg (deflated 1%)
  adding: content/data/northern cardinal/40774558663_069d1441e1_c.jpg 

In [9]:
from google.colab import files
uploaded = files.download('data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>